
# Module 2, Session 2 — Practical Exercises (BFS, DFS, Dijkstra)

1. **Trace the Search (Conceptual)** — Determine expansion orders for BFS and DFS on a small graph  
2. **Implement BFS** — Shortest path in an unweighted Armenia mini‑map  
3. **Implement DFS** — Depth‑first path on the same map  
4. **Dijkstra’s Algorithm** — Shortest weighted paths + explicit best route reconstruction



## Exercise 1 — Trace the Search (Conceptual)

**Graph** (neighbors expanded in alphabetical order):

- S → A, B  
- A → C, D  
- B → E, F  
- D → G

**Goal:** List the order of *nodes expanded*:
- **Breadth‑First Search (BFS)**
- **Depth‑First Search (DFS)**

**Key ideas**
- **BFS** explores **level by level** using a **queue (FIFO)**  
- **DFS** explores **as deep as possible** first using a **stack (LIFO)**


In [7]:

from collections import deque

# Conceptual graph
concept_graph = {
    "S": ["A", "B"],
    "A": ["C", "D"],
    "B": ["E", "F"],
    "C": [],
    "D": ["G"],
    "E": [],
    "F": [],
    "G": []
}

def bfs_expansion_order(graph, start, goal):
    visited = set()
    q = deque([start])
    order = []
    while q:
        u = q.popleft()
        if u in visited:
            continue
        visited.add(u)
        order.append(u)
        if u == goal:
            break
        for v in sorted(graph.get(u, [])):  # alphabetical expansion
            if v not in visited:
                q.append(v)
    return order

def dfs_expansion_order(graph, start, goal):
    visited = set()
    stack = [start]
    order = []
    while stack:
        u = stack.pop()
        if u in visited:
            continue
        visited.add(u)
        order.append(u)
        if u == goal:
            break
        # push neighbors in reverse alphabetical so alpha-first is popped next
        for v in sorted(graph.get(u, []), reverse=True):
            if v not in visited:
                stack.append(v)
    return order

bfs_order = bfs_expansion_order(concept_graph, "S", "G")
dfs_order = dfs_expansion_order(concept_graph, "S", "G")
print("BFS nodes expanded:", bfs_order)
print("DFS nodes expanded:", dfs_order)


BFS nodes expanded: ['S', 'A', 'B', 'C', 'D', 'E', 'F', 'G']
DFS nodes expanded: ['S', 'A', 'C', 'D', 'G']



**Expected conceptual result**  
- **BFS:** `S, A, B, C, D, E, F, G`  
- **DFS:** `S, A, C, D, G`

**Why this happens**
- BFS processes `S` first, then all of `S`’s children (`A`, `B`), then their children, etc.  
- DFS goes `S → A → C` (as deep as possible), backtracks, then `D → G`.



## Exercise 2 — Implement Breadth‑First Search (BFS)

We’ll use an **unweighted** Armenia mini‑map. BFS returns the **shortest path by number of edges**.

**Graph (adjacency list):**

```text
Yerevan:  Gyumri, Sevan
Gyumri:   Yerevan, Vanadzor
Sevan:    Yerevan, Dilizhan
Vanadzor: Gyumri, Dilizhan
Dilizhan: Sevan, Vanadzor
```

**Approach**
- Keep a `queue` of **paths** (lists of nodes).  
- Pop from the left; if the last node is the goal, return the path.  
- Otherwise, extend the path to each unvisited neighbor and push back to the queue.


In [8]:

from collections import deque

graph_unweighted = {
    "Yerevan": ["Gyumri", "Sevan"],
    "Gyumri": ["Yerevan", "Vanadzor"],
    "Sevan": ["Yerevan", "Dilizhan"],
    "Vanadzor": ["Gyumri", "Dilizhan"],
    "Dilizhan": ["Sevan", "Vanadzor"]
}

def bfs_path(graph, start, goal):
    if start == goal:
        return [start]
    visited = set([start])
    q = deque([[start]])
    while q:
        path = q.popleft()
        node = path[-1]
        if node == goal:
            return path
        # alphabetical neighbor order for determinism
        for nbr in sorted(graph.get(node, [])):
            if nbr not in visited:
                visited.add(nbr)
                q.append(path + [nbr])
    return None

print("BFS Yerevan → Dilizhan:", bfs_path(graph_unweighted, "Yerevan", "Dilizhan"))


BFS Yerevan → Dilizhan: ['Yerevan', 'Sevan', 'Dilizhan']



**Explanation**
- BFS guarantees the fewest edges. With our map, a typical result is:  
  `Yerevan → Sevan → Dilizhan`  
- We store complete paths in the queue so that when we reach the goal, we can return the route immediately.



## Exercise 3 — Implement Depth‑First Search (DFS)

DFS explores one branch as deep as possible before backtracking. It **does not** guarantee the fewest edges.

**Key difference from BFS:** we use a **stack** instead of a queue.


In [9]:

def dfs_path(graph, start, goal):
    if start == goal:
        return [start]
    visited = set([start])
    stack = [[start]]
    while stack:
        path = stack.pop()
        node = path[-1]
        if node == goal:
            return path
        # push neighbors in reverse alphabetical so alpha-first is popped next
        for nbr in sorted(graph.get(node, []), reverse=True):
            if nbr not in visited:
                visited.add(nbr)
                stack.append(path + [nbr])
    return None

print("DFS Yerevan → Dilizhan:", dfs_path(graph_unweighted, "Yerevan", "Dilizhan"))


DFS Yerevan → Dilizhan: ['Yerevan', 'Gyumri', 'Vanadzor', 'Dilizhan']



**Explanation**
- DFS may return a longer path than BFS (e.g., `Yerevan → Gyumri → Vanadzor → Dilizhan`).  
- The path depends on neighbor ordering and the order of pushes/pops on the stack.



## Exercise 4 — Dijkstra’s Algorithm (Weighted Shortest Paths)

Now we’ll add **weights (kilometers)** to edges and compute the **shortest weighted paths** from a start node.

**High‑level steps**
1. Initialize distances `dist` to `∞` except `dist[start] = 0`  
2. Repeatedly pick the **unvisited node with smallest `dist`**  
3. **Relax** outgoing edges: if `dist[u] + w(u,v) < dist[v]`, update `dist[v]` and set `prev[v] = u`  
4. After the loop, use `prev` to reconstruct the best path to any goal


In [10]:

import math

graph_weighted = {
    "Yerevan": {"Gyumri": 120, "Sevan": 70},
    "Gyumri": {"Yerevan": 120, "Vanadzor": 60},
    "Sevan": {"Yerevan": 70, "Dilizhan": 35},
    "Vanadzor": {"Gyumri": 60, "Dilizhan": 40},
    "Dilizhan": {"Sevan": 35, "Vanadzor": 40}
}

def dijkstra(graph, start):
    nodes = list(graph.keys())
    dist = {n: math.inf for n in nodes}
    prev = {n: None for n in nodes}
    visited = set()
    dist[start] = 0

    while len(visited) < len(nodes):
        # pick unvisited node with smallest distance
        u = None
        best = math.inf
        for n in nodes:
            if n not in visited and dist[n] < best:
                best = dist[n]
                u = n
        if u is None:
            break
        visited.add(u)

        for v, w in graph.get(u, {}).items():
            if v in visited:
                continue
            alt = dist[u] + w
            if alt < dist[v]:
                dist[v] = alt
                prev[v] = u

    return dist, prev

def reconstruct_path(prev, start, goal):
    path = []
    cur = goal
    while cur is not None:
        path.append(cur)
        if cur == start:
            break
        cur = prev[cur]
    path.reverse()
    return path if path and path[0] == start else None

dist, prev = dijkstra(graph_weighted, "Yerevan")
best_path = reconstruct_path(prev, "Yerevan", "Dilizhan")
print("Dijkstra distances from Yerevan:", dist)
print("Best weighted path Yerevan → Dilizhan:", best_path)


Dijkstra distances from Yerevan: {'Yerevan': 0, 'Gyumri': 120, 'Sevan': 70, 'Vanadzor': 145, 'Dilizhan': 105}
Best weighted path Yerevan → Dilizhan: ['Yerevan', 'Sevan', 'Dilizhan']



**Explanation**
- `dist` stores the best known distance from the start to each node  
- `prev` lets us reconstruct the route by backtracking from the goal  
- With our example weights, the best route Yerevan → Dilizhan is through **Sevan**
